# MIMIC-IV ICU Module Preprocessing
This Jupyter Notebook is used to preprocess the chartevents, labevents and outputevents from ICU module.

## Requirements and Installation
- Recommended installation: Devcontainer vscode extension
- Open the repository in the Devcontainer
- Move the mimic-iv dataset into the `data/external/mimic-iv-2.2` folder
- All requirements should automatically installed

IPython extension autoreload can be used to reload modules before executing user code.
https://ipython.org/ipython-doc/3/config/extensions/autoreload.html

In [1]:
%load_ext autoreload
%autoreload 2

As configuation management framework, we use Hydra (see https://hydra.cc/)

In [2]:
from hydra import compose, initialize

import hydra
from hydra.core.global_hydra import GlobalHydra

from src.data.datamodule import DataModule
from src.data import utils as utils

In [3]:
GlobalHydra().instance().clear()
initialize(version_base=None, config_path=".", job_name="train_config")

hydra.initialize()

Have a look into the `train_config.yaml` file. All configuration variables are stored there.
You can always override the configurations by using the `overrides` parameter in the `compose` function.

Important configuration parameters:
- `dataset.datamodule.props.path`: dataset path
- `data_type`: Dataformat type: tabular or timeseries
- `feature_list`: List of features that are used from the icu module

In [4]:
cfg_tab = compose(config_name="train_config", overrides=["data_type=tabular"])

Instantiation of the Datamodule

In [5]:
datamodule: DataModule = hydra.utils.instantiate(cfg_tab.dataset.datamodule)

The `datamodule.pipeline` generates a dataframe with the given ``feature_list`` and the ``target`` as tabular or timeseries dataframe.
In the following dataframe, for each feature the featurevents are aggregated with different functions like max, mean, min, and std over the `cut_off_time`.

In [6]:
dataset = datamodule.pipeline(
    cfg_tab.data_type, cfg_tab.feature_list, cfg_tab.target
)
dataset.head()

building MIMIC-IV tabular features


,stay_id,Arterial Blood Pressure diastolic_max,Arterial Blood Pressure diastolic_mean,Arterial Blood Pressure diastolic_min,Arterial Blood Pressure diastolic_std,Arterial Blood Pressure mean_max,Arterial Blood Pressure mean_mean,Arterial Blood Pressure mean_min,Arterial Blood Pressure mean_std,Arterial Blood Pressure systolic_max,...,Vti High_min,Current Dyspnea Assessment_min,Respiratory Rate (Set)_max,Fspn High_mean,Glucose finger stick (range 70-100)_min,Current Dyspnea Assessment_max,Pain Level Response_std,Richmond-RAS Scale_max,Total PEEP Level_min,icu_los
0,31269608,58.0,53.818182,49.0,3.250175,89.0,81.909091,75.0,4.700097,147.0,...,NaN,0.0,NaN,NaN,182.0,0.0,0.000000,0.0,NaN,7.702512
1,37509585,75.0,62.888889,47.0,7.653441,103.0,85.222222,63.0,11.175370,162.0,...,NaN,0.0,NaN,NaN,127.0,0.0,NaN,0.0,NaN,5.452662
2,32554129,74.0,58.250000,49.0,7.827303,101.0,80.062500,66.0,10.871791,143.0,...,NaN,0.0,NaN,NaN,100.0,0.0,NaN,0.0,NaN,0.872685
3,31338022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,84.0,2.0,0.000000,0.0,NaN,3.766725
4,32145159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,166.0,NaN,1.407886,1.0,NaN,1.037106


In [7]:
dataset.describe()

,stay_id,Arterial Blood Pressure diastolic_max,Arterial Blood Pressure diastolic_mean,Arterial Blood Pressure diastolic_min,Arterial Blood Pressure diastolic_std,Arterial Blood Pressure mean_max,Arterial Blood Pressure mean_mean,Arterial Blood Pressure mean_min,Arterial Blood Pressure mean_std,Arterial Blood Pressure systolic_max,...,Vti High_min,Current Dyspnea Assessment_min,Respiratory Rate (Set)_max,Fspn High_mean,Glucose finger stick (range 70-100)_min,Current Dyspnea Assessment_max,Pain Level Response_std,Richmond-RAS Scale_max,Total PEEP Level_min,icu_los
count,1.330000e+02,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,...,54.000000,49.000000,52.000000,54.000000,101.000000,49.000000,52.000000,131.000000,44.000000,133.000000
mean,3.512130e+07,76.363636,58.902884,46.345455,7.547830,107.618182,78.728014,58.890909,12.570543,143.963636,...,214.923889,0.183673,17.961538,24.717041,127.594059,0.775510,1.039797,0.015267,6.663636,3.033942
std,2.821390e+06,12.831333,7.008558,8.016691,2.973076,47.437480,14.352244,12.385285,17.033965,18.345414,...,421.546876,0.666879,4.830303,11.852365,47.111820,1.747204,0.940737,1.451708,3.141255,2.710199
min,3.005745e+07,53.000000,43.304348,23.000000,1.246423,79.000000,49.260870,14.000000,2.563480,112.000000,...,1.000000,0.000000,12.000000,10.000000,66.000000,0.000000,0.000000,-5.000000,5.000000,0.023727
25%,3.255413e+07,67.500000,54.114103,43.000000,5.633243,89.000000,73.774359,55.000000,7.271386,131.500000,...,1.200000,0.000000,14.000000,14.250000,94.000000,0.000000,0.283473,0.000000,5.000000,1.161019
50%,3.514680e+07,75.000000,59.037037,47.000000,7.369452,99.000000,77.125000,60.000000,9.065201,143.000000,...,1.400000,0.000000,16.000000,21.250000,115.000000,0.000000,0.865544,0.000000,5.000000,2.022963
75%,3.726758e+07,83.500000,62.982180,51.500000,8.568108,105.500000,80.731852,66.000000,11.514194,160.000000,...,1.900000,0.000000,20.500000,34.750000,153.000000,1.000000,1.500000,0.000000,6.500000,4.446551
max,3.988077e+07,120.000000,74.428571,64.000000,18.167606,354.000000,164.333333,83.000000,126.229421,180.000000,...,1500.000000,4.000000,35.000000,55.000000,305.000000,8.000000,4.242641,4.000000,18.000000,13.214676


The following cells, override the data_type with `timeseries` to generate a dataframe with timeseries data. Each sample represents a measurement at timestamp `abs_event_time` for the patient with a `subject_id` and `stay_id`.

In [8]:
cfg_time = compose(config_name="train_config", overrides=["data_type=timeseries"])

In [9]:
datamodule_time: DataModule = hydra.utils.instantiate(cfg_time.dataset.datamodule)

In [10]:
dataset_time = datamodule.pipeline(
    cfg_time.data_type, cfg_time.feature_list, cfg_time.target
)
dataset_time.head()

building MIMIC-IV timeseries features


,abs_event_time,stay_id,Activity / Mobility (JH-HLM),Apnea Interval,Arterial Blood Pressure Alarm - High,Arterial Blood Pressure Alarm - Low,Arterial Blood Pressure diastolic,Arterial Blood Pressure mean,Arterial Blood Pressure systolic,Current Dyspnea Assessment,...,Temperature,Urea Nitrogen,White Blood Cells,pCO2,pH,pO2,Time in the ICU,icu_los,icu_los_hour_int,remaining_icu_los_hour
subject_id,,,,,,,,,,,,,,,,,,,,,
10023117,1970-01-01 01:00:00,30057454,NaN,NaN,NaN,NaN,64.800000,75.0,96.800000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4.446551,106,105.0
10023117,1970-01-01 02:00:00,30057454,NaN,NaN,NaN,NaN,62.333333,72.0,93.333333,NaN,...,NaN,45.0,12.7,NaN,NaN,NaN,2.0,4.446551,106,104.0
10023117,1970-01-01 03:00:00,30057454,NaN,NaN,NaN,NaN,64.000000,76.0,99.000000,NaN,...,NaN,45.0,12.7,45.0,7.4,76.0,3.0,4.446551,106,103.0
10023117,1970-01-01 04:00:00,30057454,NaN,NaN,NaN,NaN,67.000000,78.0,102.000000,NaN,...,NaN,45.0,12.7,45.0,7.4,76.0,4.0,4.446551,106,102.0
10023117,1970-01-01 05:00:00,30057454,NaN,NaN,NaN,NaN,67.000000,79.0,105.000000,NaN,...,NaN,45.0,12.7,45.0,7.4,76.0,5.0,4.446551,106,101.0


In [11]:
dataset_time.describe()

,abs_event_time,stay_id,Activity / Mobility (JH-HLM),Apnea Interval,Arterial Blood Pressure Alarm - High,Arterial Blood Pressure Alarm - Low,Arterial Blood Pressure diastolic,Arterial Blood Pressure mean,Arterial Blood Pressure systolic,Current Dyspnea Assessment,...,Temperature,Urea Nitrogen,White Blood Cells,pCO2,pH,pO2,Time in the ICU,icu_los,icu_los_hour_int,remaining_icu_los_hour
count,2916,2.916000e+03,1254.000000,1124.000000,947.000000,946.000000,1133.000000,1149.000000,1133.000000,978.000000,...,399.000000,1617.000000,1624.000000,1198.000000,1441.000000,1198.000000,2916.000000,2916.000000,2916.000000,2916.000000
mean,1970-01-01 12:18:36.049382716,3.510506e+07,2.199761,20.867438,141.800422,84.139535,59.068482,78.056050,115.460041,0.391616,...,36.809023,26.427953,15.295813,41.756260,7.064860,124.605593,12.310014,3.227520,76.927298,64.617284
min,1970-01-01 00:00:00,3.005745e+07,1.000000,20.000000,90.000000,45.000000,23.000000,-23.000000,58.000000,0.000000,...,35.300000,5.000000,0.400000,20.000000,5.000000,18.000000,0.000000,0.023727,0.000000,-2.000000
25%,1970-01-01 06:45:00,3.250612e+07,2.000000,20.000000,130.000000,85.000000,52.857143,69.000000,102.000000,0.000000,...,36.400000,13.000000,8.800000,35.000000,7.230000,64.000000,6.750000,1.207407,28.000000,18.000000
50%,1970-01-01 12:00:00,3.512824e+07,2.000000,20.000000,140.000000,90.000000,59.000000,75.571429,113.000000,0.000000,...,36.700000,19.000000,11.300000,41.000000,7.360000,96.500000,12.000000,2.199653,52.000000,41.000000
75%,1970-01-01 18:00:00,3.726758e+07,2.000000,20.000000,160.000000,90.000000,65.000000,84.000000,127.000000,0.000000,...,37.200000,34.000000,17.100000,45.000000,7.410000,154.000000,18.000000,4.844086,116.000000,98.000000
max,1970-01-02 00:00:00,3.988077e+07,8.000000,60.000000,190.000000,120.000000,120.000000,354.000000,172.000000,8.000000,...,40.400000,108.000000,116.100000,112.000000,7.540000,492.000000,24.000000,13.214676,317.000000,316.000000
std,NaN,2.813006e+06,0.870144,5.000378,22.301634,14.152517,10.486620,23.484846,19.130344,1.263215,...,0.843788,19.423974,14.802951,10.242194,0.638164,92.633184,6.779237,2.702597,64.860871,64.856456
